---
title: "2023_Kronensicherung_Plesse_Analyse"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Analyse

## Kombinierte Analyse LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten


### IMPORT: Packages

In [ ]:
# Struktur & Typen
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Zum Vereinheitlichen von Strings

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

# Statistik
from scipy.stats import linregress, f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as mc

In [ ]:
# Eigene Module und Funktionen
from kj_core.utils.latex_export import (
    generate_latex_table,
    generate_grouped_latex_tables,
    save_latex_table,
    build_data_dict_df
)
from kj_core.utils.labeling import (
    get_label_from_dict,
    get_color_dict
)
from kj_core import (
    CoreConfig,
    PlotManager,
    get_logger
)

# Projekteinstellungen
from project_config import (
    working_directory,
    data_export_directory,
    latex_export_directory,
    filename_clean_dataset,
    filename_clean_data_dict
)

### IMPORT: Manager Instanzen

In [ ]:
logger = get_logger(__name__)
CONFIG = CoreConfig(working_directory=f"{working_directory}/combined")
PLOT_MANAGER = PlotManager(CONFIG)

## IMPORT: Daten Import

In [ ]:
# Dateien laden
df = pd.read_feather(data_export_directory / filename_clean_dataset)

with open(data_export_directory / filename_clean_data_dict, "r", encoding="utf-8") as f:
    data_dict = json.load(f)

## ANALYSE: Explorative Datenanalyse

In [ ]:
df.head(10)

In [ ]:
df.columns

### COMBINED: Definition von Darstellungsstandards
Festlegen von Farbcodes für einheitliche Darstellung von Sensoren und Behandlungsvarianten für alle nachfolgenden Plots.

In [ ]:
color_palette = PLOT_MANAGER.color_palette_list

# Für die Spalte "treatment":
treatment_color_dict = get_color_dict(df, "treatment", PLOT_MANAGER.color_palette_list)
# Für die Spalte "sensor_name":
sensor_color_dict = get_color_dict(df, "sensor_name", PLOT_MANAGER.color_palette_list)

### LATEX-EXPORT: Latex-Export von Daten für Anhang

In [ ]:
variables_2 = ['id', 'sensor_name', 'treatment', 'm_amplitude', 'm_amplitude_2', 'initial_amplitude', 'damping_coeff', 'damping_ratio', 'frequency_damped', 'frequency_undamped', 'y_shift', 'pearson_r', 'nmae']

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables_2]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables_2})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Schwingungsparameter vollständig",
    column_format="lrl|rrr|rr|rr|r|rr",
    group_by="treatment",
    latex_export_directory=latex_export_directory
)

### PTQ: Analyse der Schwingungsparameter

In diesem Abschnitt werden die Schwingungsparameter statistisch ausgewertet. Ziel ist es, den Einfluss verschiedener Behandlungsvarianten (treatment) auf die gemessenen Schwingungsparameter zu untersuchen und dabei auch den potenziellen Einfluss der Vorspannung (rope_release) und Sensorposition (sensor_name) zu berücksichtigen.


In [ ]:
variables = [
    'm_amplitude', 
    'm_amplitude_2',
    'max_strain',
    'max_compression',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    #'nrmse', 
    'nmae'
]

#### Systematischer Einfluss der Sensorposition

Ziel: Visuell erkennen, ob unterschiedliche Sensoren konsistent andere Werte liefern.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="sensor_name", y=var, data=df, dodge=True)
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Sensoren auf {get_label_from_dict(var, data_dict, use_titel=True)}")
    plt.xlabel("Sensorname")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    #plt.show(
    PLOT_MANAGER.save_plot(fig, filename=f"effect_sensor_{var}", subdir="ptq_osc/sensor_only")

Ziel: Feststellen, ob die Variation durch unterschiedliche Sensorpostion relativ zur behandlungsbedingten Variation relevant ist.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict,  dodge=True)
    # Stripplot: Punkte zur Veranschaulichung der Verteilung
    sns.stripplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict, dodge=True, alpha=1, jitter=True, size=5, legend=False)
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Behandlung auf {get_label_from_dict(var, data_dict, use_titel=True)} gruppiert über Sensoren")
    plt.xlabel("Sensorname")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_sensor_treatment_{var}", subdir="ptq_osc/sensor_treatment")

Ziel: Feststellen, ob die Variation durch unterschiedliche Behandlungen relativ zur sensorbedingten Variation relevant ist.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name')
    sns.stripplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name', dodge=True, alpha=1, jitter=True, size=5, legend=False)
    plt.title(f"Einfluss der Sensoren auf {get_label_from_dict(var, data_dict, use_titel=True)} gruppiert über Behandlung")
    plt.xlabel("Behandlungsvariante")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_sensor_{var}", subdir="ptq_osc/treatment_sensor")

##### Statistische Untersuchung des Effektes der Sensorposition innerhalb einer Beobachtung

In [ ]:
# 1. Deskriptive Statistiken pro Sensor
summary_by_sensor = df.groupby('sensor_name', observed=False)[variables].describe()
summary_by_sensor

In [ ]:
# 3. ANOVA: Teste, ob die Unterschiede zwischen den Sensoren signifikant sind
print("\nANOVA-Tests für Unterschiede zwischen den Sensoren:")
for var in variables:
    # Fit eines linearen Modells mit sensor_name als Prädiktor
    model = smf.ols(formula=f"{var} ~ C(sensor_name)", data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(f"\nANOVA für {var}:")
    print(anova_table)

##### Optimaler Ansatz: Versuche Mixed-Linear Model

Die Daten sind hierarchisch: mehrere Messungen (vier Sensoren) pro Beobachtungseinheit (`id`). Ein Mixed-Effects Modell könnte diese Struktur abbilden, indem zufällige Effekte für `id` und feste Effekte für `treatment` sowie `sensor_name` berücksichtigt werden. Zusätzlich kann `rope_release` als Kovariate eingeführt werden. Dieser Ansatz wären theoretisch optimal, aber aufgrund der geringen Stichprobengröße und der komplexen Datenstruktur treten Konvergenzprobleme auf.

In [ ]:
# Ergebnisse als Dictionary speichern
results_dict = {}

for var in variables:
    model = smf.mixedlm(
        f"{var} ~ C(treatment) + C(sensor_name) + rope_release", 
        data=df, 
        groups=df["id"]
    )
    result = model.fit()
    print(f"\n### Ergebnisse für {var}:")
    print(result.summary())
    print("\n")

Um dennoch aussagekräftige Ergebnisse zu erhalten, wird ein zweistufiges Verfahren genutzt.
Zwei Varianten:
- Mittlung über alle Sensoren innerhalb eine Beobachtung. Die Variation zwischen den Sensoren bleibt unberücksichtigt und verfälscht ggf. die Ergebnisse. Vorteil: Sehr einfach und stabil
- Werte um Sensor-Effekt manuell adjustieren und dann aggregieren wie zuvor. Vorteile: Sensoren verfälschen nicht die Ergebnisse. Nachteil ist die höhere Komplexität und das die Gesamtvariation der Modell am ende verfälsch ist.

##### Alternative 1: Vereinfachtes Vorgehen durch Aggregation (Mittelwert)

In [ ]:
df_mean = df.groupby("id").agg({var: "mean" for var in variables}).reset_index()
# Benenne die Spalten um, damit klar ist, dass es sich um Mittelwerte handelt.
df_mean = df_mean.rename(columns={var: f"{var}_mean" for var in variables})

# Füge zusätzliche Variablen (z.B. treatment und rope_release) hinzu.
treatment_df = df[["id", "treatment", "rope_release"]].drop_duplicates()
df_mean = df_mean.merge(treatment_df, on="id")
df_mean.describe()

##### Alternative 2: Werte um Sensor-Effekt und Vorspannung Adjustieren und dann Aggregieren (Mittelwert)

Nicht jeder Parameter wird durch `rope_release` beeinflusst. Nur für jene Parameter, bei denen ein signifikanter Einfluss von `rope_release` festgestellt wird, soll dieser Effekt herausgerechnet werden. Auf diese Weise entstehen "bereinigte" Werte, in denen der lineare Einfluss von `rope_release` entfernt ist.

In [ ]:
# Definiere die Liste der Variablen, bei denen rope_release entfernt werden soll
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']
model_dict = {}
# Erstelle eine Kopie des Original-DataFrames
df_copy = df.copy()

for var in variables:
    try:
        # Wähle die Modellformel abhängig davon, ob rope_release berücksichtigt werden soll
        if var in relevant_vars_rope_release:
            formula = f"{var} ~ C(sensor_name) + rope_release"
        else:
            formula = f"{var} ~ C(sensor_name)"
            
        model = smf.ols(formula, data=df).fit()
        
        # Speichere das Modell im Dictionary (so hast du per Variable die komplette Summary)
        model_dict[var] = model
        # Berechne adjustierte Werte:
        # y_adj = y - (fitted effect - globaler Intercept)
        df_copy[f"{var}_adj"] = df_copy[var] - (model.fittedvalues - model.params['Intercept'])
    except Exception as e:
        print(f"Fehler bei {var}: {e}")

# Aggregiere über die ID – hier wird der Mittelwert der adjustierten Werte berechnet.
df_adj = df_copy.groupby("id").agg({f"{var}_adj": "mean" for var in variables}).reset_index()
# Ergänze zusätzliche Variablen (z.B. treatment und rope_release)
treatment_df = df[["id", "treatment", "rope_release"]].drop_duplicates()
df_adj = df_adj.merge(treatment_df, on="id")

# Ausgabe der deskriptiven Statistik
print(df_adj.describe())


##### Zusammenführen beider Alternativen für Vergleich

In [ ]:
# --- Zusammenführen beider Ansätze ---
# Beide DataFrames enthalten nun jeweils den Mittelwert pro ID:
# - df_mean: direkte Mittelwerte mit Suffix _mean
# - df_adj: sensor-adjustierte Mittelwerte mit Suffix _adj
grouped_df = pd.merge(df_mean, df_adj, on=["id", "treatment", "rope_release"], how="inner")
grouped_df

Plotten und Vergleich beider Ergebnisse

In [ ]:
for var in variables:
    # Transformiere die Daten in das Long-Format: 
    # Es werden die beiden Spalten f"{var}_mean" und f"{var}_adj" unter einer neuen Spalte "aggregation" zusammengeführt,
    # und die Werte in der Spalte "value" gespeichert.
    df_long = pd.melt(
        grouped_df,
        id_vars=["id", "treatment", "rope_release"],
        value_vars=[f"{var}_mean", f"{var}_adj"],
        var_name="aggregation",
        value_name="value"
    )
    # Kürze die Beschriftungen, sodass nur "mean" bzw. "adj" übrig bleiben
    df_long["aggregation"] = df_long["aggregation"].str.replace(f"{var}_", "", regex=False)
    
    # Erstelle den Plot: Boxplots nach Behandlung, getrennt nach Aggregationsmethode
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y="value", hue="aggregation", data=df_long, dodge=True)
    
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Behandlung auf {get_label_from_dict(var, data_dict, use_titel=True)}")
    plt.xlabel("Behandlung")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    
    # Speichere den Plot (ersetze PLOT_MANAGER.save_plot durch plt.show() oder deine eigene Funktion, falls nötig)
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_{var}", subdir="ptq_osc/treatment_comparison")


In [ ]:
def extract_model_info(model):
    """
    Extrahiert zentrale Kennzahlen aus einem OLS-Modell.
    Liefert ein Dictionary mit R-squared, Adj. R-squared, sowie den Koeffizienten,
    Standardfehler, t-Werten und p-Werten für alle Parameter.
    """
    info = {
        "R²": model.rsquared,
        "adj_R²": model.rsquared_adj,
    }
    # Iteriere über alle Parameter im Modell
    for param in model.params.index:
        info[f"coef_{param}"] = model.params[param]
        info[f"std_err_{param}"] = model.bse[param]
        info[f"t_{param}"] = model.tvalues[param]
        info[f"p_{param}"] = model.pvalues[param]
    return info

In [ ]:
# Erstelle eine Liste, in der pro Variable das extrahierte Dictionary gespeichert wird.
model_info_list = []
for var, model in model_dict.items():
    info = extract_model_info(model)
    info['variable'] = var  # Variable als Kennung hinzufügen
    model_info_list.append(info)

# Konvertiere die Liste in einen DataFrame und setze "variable" als Index
df_model_info = pd.DataFrame(model_info_list).set_index("variable")
df_model_info.index.name = "Statistik"
df_model_info = df_model_info.T

In [ ]:
def format_value(x):
    # Prüfe, ob der Wert numerisch ist
    if isinstance(x, (int, float, np.number)):
        if abs(x) < 0.0001:
            return f"{x:.2e}"
        else:
            return f"{x:.2f}"
    return x

# Wende die Funktion auf alle Zellen von df_model_info an
df_model_info = df_model_info.applymap(format_value)
df_model_info

In [ ]:
# Manuelles Escapen der Unterstriche im Index
df_model_info.index = df_model_info.index.astype(str).str.replace('(sensor_name)[T.', r'(sensor)[')
df_model_info.index = df_model_info.index.astype(str).str.replace('_', r'\_')


# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_model_info = df_model_info.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})

column_format = "l" + "r" * (len(df_model_info.columns))
latex_string = df_model_info.to_latex(index=True, escape=False, float_format="%.2f", column_format=column_format)

# Definiere Beschriftung und lange Beschriftung (Caption)
caption = "Feldversuch 2: Ergebnisse, Zusammenfassung der OLS-Modelle"
caption_long = "Übersicht über zentrale Kennzahlen der OLS-Modelle: R², Adjusted R², Koeffizienten, Standardfehler, t-Werte und p-Werte für alle Parameter."

# Funktion zum Speichern der LaTeX-Tabelle (du passt diese Funktion ggf. an deine Gegebenheiten an)
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Auf dieser Basis können einfache OLS-Modelle geschätzt werden, z. B. `Parameter ~ C(treatment)`. Diese Modelle sind einfacher und sollten stabil konvergieren.

#### Systematischer Einfluss der Behandlungsvariante

In [ ]:
grouped_df

In [ ]:
df_adj.columns = df_adj.columns.str.replace('_adj$', '', regex=True)
desired_order = ['id', 'treatment', 'rope_release']
other_columns = [col for col in df_adj.columns if col not in desired_order]
new_order = desired_order + other_columns
df_adj = df_adj[new_order]
df_adj

In [ ]:
import pandas as pd
from tabulate import tabulate
import re

# Liste der Variablen, die in die Tabelle übernommen werden sollen
variables_spec = [
    'm_amplitude_2',
    'max_compression',
    'damping_coeff',
    'damping_ratio',
    'frequency_damped',
    'frequency_undamped',
]

# Umbenennung der Spalten für LaTeX-Notation
column_rename_map = {
    'm_amplitude_2': r'$mA_2$',
    'max_compression': r'$\text{max\_C}$',
    'damping_coeff': r'$\delta$',
    'damping_ratio': r'$D$',
    'frequency_damped': r'$f_d$',
    'frequency_undamped': r'$f_0$',
}

# Funktion zur Erstellung der Modellgüte-Kennzahlen für alle Variablen
def create_model_metrics_table(variables, results):
    metrics_data = {
        "Kennzahl": ["R²", "Adj. R²", "F-St.", "AIC", "N"]
    }

    for var in variables:
        col_name = column_rename_map.get(var, var)  # Verwende gekürzten Namen, falls vorhanden
        if var not in results or 'error' in results[var]:
            metrics_data[col_name] = ["n/a"] * 5
        else:
            model = results[var]['model']
            metrics_data[col_name] = [
                f"{model.rsquared:.4f}",
                f"{model.rsquared_adj:.4f}",
                f"{model.fvalue:.4f}",
                f"{model.aic:.4f}",
                f"{model.nobs:.0f}"
            ]

    # Erstelle die Tabelle mit tabulate
    metrics_df = pd.DataFrame(metrics_data)
    return tabulate(
        metrics_df,
        headers="keys",
        tablefmt="latex_raw",
        floatfmt=".4f",
        showindex=False)

# Funktion zur Erstellung einer LaTeX-Tabelle für die Koeffizienten einer Variable
def create_latex_table_for_variable(var, results):
    if var not in results:
        return f"%% Keine Ergebnisse für Variable {var} vorhanden."
    
    model_result = results[var]
    if 'error' in model_result:
        return f"%% Fehler beim Anpassen des Modells für {var}: {model_result['error']}"
    
    model = model_result['model']
    summary = model.summary2().tables[1]  # Zugriff auf die Tabelle der Koeffizienten

    # Erstelle eine LaTeX-Tabelle mit tabulate für die Koeffizienten
    latex_table = tabulate(
        summary,
        headers=summary.columns,
        tablefmt="latex_booktabs",
        floatfmt=".4f",
        showindex=True
    )

    # Escape Unterstriche in der Variable für LaTeX
    escaped_var = re.sub(r'_', r'\_', var)
    shortened_var = column_rename_map.get(var, escaped_var)

    # Füge die LaTeX-Caption zur Tabelle hinzu
    return f"""
\\begin{{table}}[ht]
    \\centering
    \\caption{{Modellzusammenfassung für {escaped_var} ({shortened_var})}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {latex_table}
    \\end{{adjustbox}}
\\end{{table}}
"""

# Erstelle die Modellgüte-Tabelle für alle Variablen
latex_metrics_table = create_model_metrics_table(variables_spec, results)
print("""
\\begin{table}[ht]
    \\centering
    \\caption{Modellgüte für alle Variablen}
    \\begin{adjustbox}{max width=\\textwidth}
""")
print(latex_metrics_table)
print("""
    \\end{adjustbox}
\\end{table}

\\vspace{1cm}
""")

# Erstelle und print die LaTeX-Tabellen für die Koeffizienten jeder Variable
for var in variables_spec:
    latex_output = create_latex_table_for_variable(var, results)
    print(latex_output)

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables), 2, (len(variables) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_with_rope_release", subdir="combined")


Für die betroffenen Parameter wird der Einfluss von `rope_release` mithilfe der bereits angepassten Modelle (`Parameter ~ C(treatment) + rope_release`) entfernt. Dazu werden Vorhersagen für einen konstanten `rope_release`-Wert (den Mittelwert) berechnet und mit den tatsächlichen Werten verglichen. Die daraus resultierenden bereinigten Werte sind frei von Variation, die auf `rope_release` zurückzuführen wäre.

In [ ]:
grouped_adjusted_df = grouped_df.copy()

# Bereinigung: Wir setzen rope_release auf seinen Mittelwert
rope_mean = grouped_df["rope_release"].mean()

for var in relevant_vars_rope_release:
    # Zugehöriges Modellobjekt abrufen
    model = results[var]["model"]

    # Vorhersage mit tatsächlichen rope_release-Werten
    predicted_current = model.predict(grouped_df)

    # Vorhersage, wenn rope_release = rope_mean gesetzt wird
    df_mean_rope = grouped_df.copy()
    df_mean_rope["rope_release"] = rope_mean
    predicted_mean = model.predict(df_mean_rope)

    # Angepasste Werte berechnen:
    actual = grouped_df[var].values
    adjusted = actual + (predicted_mean - predicted_current)
    grouped_adjusted_df[var] = adjusted

In [ ]:
grouped_df.describe()

In [ ]:
grouped_adjusted_df.describe()

##### Latex Tabelle Output


In [ ]:
data_dict

In [ ]:
# DataFrame mit den relevanten Spalten erstellen
variables_latex = [
    'm_amplitude', 
    'm_amplitude_2',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    'nmae'
]
df_latex = grouped_adjusted_df[variables_latex + ['treatment']].copy()
df_latex.columns

In [ ]:


# Spaltennamen entsprechend der LaTeX-Notation umbenennen
df_latex.rename(columns=data_dict, inplace=True)

# Erstellung deskriptiver Statistiken für alle Beobachtungen
overall_stats = df_latex.describe().drop(index=['count', '25%', '75%'])
overall_stats.rename(index={'50%': 'median'}, inplace=True)

# Erstellung der deskriptiven Statistiken für jede Gruppe
grouped_stats = {
    'overall': overall_stats
}

for treatment, group in df_latex.groupby('treatment', observed=True):
    group_stats = group.describe().drop(index=['count', '25%', '75%'])
    group_stats.rename(index={'50%': 'median'}, inplace=True)
    grouped_stats[treatment] = group_stats

# Zusammenführen der Statistiken in einer Tabelle
combined_stats = pd.concat(grouped_stats, names=['Treatment'])

# LaTeX-Export des kombinierten DataFrames
df_latex_string = combined_stats.to_latex(
    escape=False,
    multirow=True,
    multicolumn=True,
    column_format="l|lrrrrrrrrr", 
    float_format="{:0.2f}".format
)

# LaTeX-Tabellencode erstellen
latex_table = f"""
\\begin{{table}}[h]
    \\centering
    \\caption{{Feldversuch 2 - Ergebnisse, Schwingungsparameter deskriptive Statistiken (Gesamt und gruppiert über Treatment), Amplituden korrigiert über \\texttt{{rope\\_release}}, 9 Beobachtung je Gruppe, jeweils Mittelwert für 4 Elastometer}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {df_latex_string}
    \\end{{adjustbox}}
    \\label{{tab:Feldversuch_2_Deskriptive_Statistiken_Schwingungsparameter}}
\\end{{table}}
"""

print(latex_table)

#### Visualisierung der bereinigten Ergebnisse

Abschließend werden die bereinigten Werte grafisch dargestellt, um die Unterschiede zwischen den Behandlungen in Abwesenheit des `rope_release`-Einflusses zu verdeutlichen. Dies zeigt, wie sich die Treatments auf die Parameter auswirken würden, wenn für alle Einheiten die gleiche mittlere Vorspannung gelten würde.

In [ ]:
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']

n_vars = len(relevant_vars_rope_release)
n_cols = 2  # Links Original, rechts angepasst
n_rows = n_vars  # Eine Zeile pro Variable

fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))

for i, var in enumerate(relevant_vars_rope_release):
    # Linke Spalte: Original (grouped_df)
    sns.boxplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,0].set_title(f"original: {var}")
    axes[i,0].set_ylabel(var)

    # Rechte Spalte: Angepasst (grouped_adjusted_df)
    sns.boxplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,1].set_title(f"adjusted : {var}")
    axes[i,1].set_ylabel(var)
    
    # Y-Limits von links holen
    y_min, y_max = axes[i,0].get_ylim()
    # Y-Limits auf rechts anwenden
    axes[i,1].set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()

PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_comparison", subdir="combined")


Schritt 2: Durchführung von Post-hoc-Tests

Festzustellen welche paarweisen Unterschiede zwischen den Treatments signifikant sind.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Wir gehen davon aus, dass grouped_adjusted_df existiert und die Spalten 'treatment' sowie die Variablen aus 'variables' enthält.

for var in variables_spec:
    # Tukey HSD Test durchführen
    # Annahme: Die Spalte 'treatment' enthält die Gruppennamen z.B. 'free', 'gefa_dynamic', 'cobra_static'
    # pairwise_tukeyhsd benötigt die abhängige Variable und die Gruppen.
    tukey_results = pairwise_tukeyhsd(endog=grouped_adjusted_df[var],
                                      groups=grouped_adjusted_df['treatment'],
                                      alpha=0.05)
    
    print(f"--- Post-Hoc Test (Tukey HSD) für Variable: {var} ---")
    print(tukey_results.summary())
    print("\n")


In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release", subdir="combined")

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release_spec", subdir="combined")

#### Vorhergesagte Werte extrahieren und Boxplots für die Sensoren erstellen

In [ ]:
# Vorhergesagte Werte aus den Modellen extrahieren
for variable in variables:
    df[f'predicted_{variable}'] = models[variable].fittedvalues

# Boxplots erstellen mit den vorhergesagten Werten
fig, axes = plt.subplots(4, 2, figsize=(8, 8))
axes = axes.flatten()

for i, variable in enumerate(variables):
    sns.boxplot(ax=axes[i], x='treatment', y=f'predicted_{variable}', data=df, palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    axes[i].set_title(f'{variable} by Treatment')
    axes[i].set_xlabel('Treatment')
    axes[i].set_ylabel(f'Predicted {variable}')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="predicted_effect_for_treatment", subdir="combined")

In [ ]:
def annotate_tukey(ax, tukey_result, significance_level=0.05):
    """
    Fügt eine Textbox mit den Tukey-Test-Ergebnissen und dem festgelegten Signifikanzniveau in den Plot ein.
    
    Parameters:
    ax (matplotlib.axes): Die Achse, auf der der Plot gezeichnet wird.
    tukey_result (TukeyHSDResults): Die Ergebnisse des Tukey HSD Tests.
    significance_level (float): Das Signifikanzniveau, standardmäßig 0.05.
    """
    # Definiere die gewünschte Reihenfolge der Vergleiche
    comparisons_order = [('free', 'gefa_dynamic'), ('free', 'cobra_static'), ('gefa_dynamic', 'cobra_static')]

    # Text für die Annotation zusammenstellen
    text_str = f"Tukey HSD Results: \n(Significance level = {significance_level:.2f})\n\n"
    
    # Durchlaufe die gewünschte Vergleichsreihenfolge
    for group1, group2 in comparisons_order:
        # Filtere die korrekte Paarung aus den Tukey-Ergebnissen
        for i in range(len(tukey_result._results_table.data[1:])):
            pair = tukey_result._results_table.data[i + 1]
            if (pair[0] == group1 and pair[1] == group2) or (pair[0] == group2 and pair[1] == group1):
                p_value = tukey_result.pvalues[i]
                significance = "*" if p_value < significance_level else "n.s."
                text_str += f"\n{group1} vs {group2}: \np = {p_value:.4f} ({significance})\n\n"
    
    # Textbox am Rand des Plots hinzufügen
    ax.annotate(text_str, xy=(1.01, 0.1), xycoords='axes fraction', va='center', ha='left')

# Einzelne Plots für jede Variable erstellen und speichern
for variable in variables:
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Boxplot für die aktuelle Variable
    sns.boxplot(ax=ax, x='treatment', y=f'predicted_{variable}', data=df, 
                palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    
    # Tukey-Test für die aktuelle Variable
    tukey_result = tukey_results[variable]
    
    # Tukey-Ergebnisse annotieren
    annotate_tukey(ax, tukey_result)
    
    ax.set_title(f'{variable} by Treatment')
    ax.set_xlabel('Treatment')
    ax.set_ylabel(f'Predicted {variable}')
    
    plt.tight_layout()
    plt.show()

    # Plot speichern
    plot_filename = f"{variable}_effect_for_treatment"
    PLOT_MANAGER.save_plot(fig, filename=plot_filename, subdir="osc_variables_box")


In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_treatment_describe = (df.drop(['id', 'ptq_sensor_name'], axis=1)
                         .groupby('treatment', observed=True)
                         .describe())

df_treatment_describe = df_treatment_describe.reset_index()
df_treatment_describe.round(2)

In [ ]:
df.dtypes

In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_sensor = (df.drop(['id', 'release_force_target', 'ls3_rope_release', 'ls3_cable_max', 'location', 'height', 'diameter', 'direction'], axis=1).
             groupby(['treatment', 'ptq_sensor_name'], observed=True).
             mean())  #.T
#df_sensor.round(4)

In [ ]:
# Gruppieren des DataFrames und Anwenden von mean für ptq_sensor_name 
df_id = ((df.drop(['ptq_sensor_name', 'location', 'height', 'diameter', 'direction'], axis=1)
          .groupby(['treatment', 'id'], observed=True)
          .mean())
         .reset_index())

df_id.round(4).head()

### Zusammenhangsanalyse für LS3 und PTQ

In [ ]:
# Auswahl der neuen Spaltennamen für die Korrelationsmatrix
columns_corr = ['ptq_m_amplitude',
                'ptq_m_amplitude_2',
                'ptq_initial_amplitude',
                'ptq_damping_coeff',
                'ptq_angular_frequency',
                'ptq_y_shift',
                'ptq_pearson_r',
                #'ptq_nrmse',
                'ptq_nmae',
                'release_force_target',
                'ls3_rope_release',
                'ls3_cable_max']
df_corr = df_id.copy()[columns_corr]

# Berechnung der Korrelationsmatrix
correlation_matrix = df_corr.corr()

# Visualisierung der Korrelationsmatrix mit Seaborn
fig1, ax = plt.subplots(figsize=(8, 8))  # Anpassen der Größe der Grafik
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=ax, annot_kws={'size': 10})

# Titel und Schriftgrößen anpassen
#plt.title('Correlation Matrix for LS3 and PTQ', fontsize=18)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_xlabel('')
ax.set_ylabel('')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig1, filename="correlation_matrix", subdir="combined")

In [ ]:
# ANOVA für 'ls3_rope_release'
model_rope_release = smf.ols('ls3_rope_release ~ treatment', data=df_id).fit()
anova_rope_release = sm.stats.anova_lm(model_rope_release, typ=2)

# ANOVA für 'ls3_cable_max'
model_cable_max = smf.ols('ls3_cable_max ~ treatment', data=df_id).fit()
anova_cable_max = sm.stats.anova_lm(model_cable_max, typ=2)

# Zusammenfassungen der Modelle
summary_rope_release = model_rope_release.summary()
summary_cable_max = model_cable_max.summary()
# Zusammenfassungen der Modelle
summary_rope_release_latex = model_rope_release.summary().as_latex()
summary_cable_max_latex = model_cable_max.summary().as_latex()

#print(summary_rope_release_latex)
#print(summary_cable_max_latex)

anova_rope_release, summary_rope_release, anova_cable_max, summary_cable_max

#### Zusammenhang Vorspannung und resultierende Lastspitzen

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude", subdir="combined")


In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude_2'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude_2', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude_2'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude 2')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude 2 [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude_2", subdir="combined")

In [ ]:
# Funktion zur Durchführung des ANOVA-Tests und Berechnung der Effektstärke (Eta Squared)
def perform_anova_and_effect_size(df: pd.DataFrame, variable: str, treatments: List[str]) -> str:
    groups = [df[df['treatment'] == treatment][variable].dropna() for treatment in treatments]
    f_stat, p_value = f_oneway(*groups)

    # Berechnung der Effektstärke (Eta Squared)
    n = sum([len(g) for g in groups])
    ss_total = sum([(x - df[variable].mean()) ** 2 for g in groups for x in g])
    eta_squared = f_stat * len(groups) / (f_stat * len(groups) + (n - len(groups)))

    # Überprüfung der Signifikanz
    significance = "*" if p_value < 0.05 else ""

    return f"{variable}: {significance}\nF-statistic = {f_stat:.2f}\np-value = {p_value:.2e}\nEta Squared = {eta_squared:.2f}"

In [ ]:
# Funktion zur Erstellung von Boxplots
def create_boxplot(df: pd.DataFrame, variable: str, group_by: str, ax: plt.Axes, color_dict: Dict[str, str], perform_stats: bool) -> None:
    valid_df = df.dropna(subset=[variable])
    sns.boxplot(x=group_by, y=variable, hue=group_by, data=valid_df, ax=ax, palette=color_dict, dodge=False)
    ax2 = ax.twinx()
    ax2.axis('off')
    if perform_stats:
        stats_text = perform_anova_and_effect_size(valid_df, variable, valid_df[group_by].unique())
        ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
    ax.set_title(f'Einfluss von {group_by} auf {variable}')
    ax.set_xlabel(group_by)
    ax.set_ylabel(variable)

# Funktion zur Erstellung kombinierter Plots
def create_combined_plot(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], num_columns: int = 3, perform_stats: bool = False) -> None:
    num_rows = len(columns) // num_columns + (len(columns) % num_columns > 0)
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(16, 4 * num_rows))
    axes = axes.flatten()

    for idx, variable in enumerate(columns):
        create_boxplot(df, variable, group_by, axes[idx], color_dict, perform_stats)

    plt.tight_layout()
    plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"combined_plot_{group_by}", subdir="combined")

# Funktion zur Erstellung einzelner Plots
def create_individual_plots(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], perform_stats: bool = False) -> None:
    for variable in columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        create_boxplot(df, variable, group_by, ax, color_dict, perform_stats)
        plt.tight_layout()
        #plt.show()
        PLOT_MANAGER.save_plot(fig, filename=f"{group_by}_{variable}", subdir="individual_plots")

In [ ]:
columns = ['ptq_m_amplitude',
           'ptq_m_amplitude_2',
           'ptq_initial_amplitude',
           'ptq_damping_coeff',
           'ptq_angular_frequency',
           'ptq_y_shift',
           'ptq_pearson_r',
           #'ptq_nrmse',
           #'ptq_nmae',
           #'release_force_target',
           'ls3_rope_release',
           'ls3_cable_max'
           ]

# Beispiel: Erstellen von Plots gruppiert nach 'treatment'
create_combined_plot(df, columns, 'treatment', treatment_color_dict, perform_stats=True)
create_individual_plots(df, columns, 'treatment', treatment_color_dict, perform_stats=True)

In [ ]:
# Beispiel: Erstellen von Plots gruppiert nach 'ptq_sensor_name'
columns = ['ptq_m_amplitude', 'ptq_m_amplitude_2', 'ptq_initial_amplitude', 'ptq_damping_coeff', 'ptq_angular_frequency', 'ptq_pearson_r']

create_combined_plot(df, columns, 'ptq_sensor_name', sensor_color_dict)
create_individual_plots(df, columns, 'ptq_sensor_name', sensor_color_dict)